In [1]:
import os
import os.path as osp
import sys
from collections import defaultdict
from PIL import Image
import cv2
import json
import numpy as np
from tqdm import tqdm 
def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

In [2]:
split = 'train'
imgsrc = '/media/allen/mass/DeepFashion/{}/image'.format(split)
annosrc = '/media/allen/mass/DeepFashion/{}/annos'.format(split)
imgpaths = sorted([osp.join(root, f) for root, _, files in os.walk(imgsrc) for f in files if 'jpg' in f or 'png' in f])

In [3]:
annos = []
for path in imgpaths:
    img = cv2.imread(path)
    fname = osp.splitext(osp.basename(path))[0]
    f = open(osp.join(annosrc, "{}.json".format(fname)))
    raw_anno = json.loads(f.readline())
    num_objs = len(raw_anno) - 2
    anno = defaultdict(dict)
    for k in range(1, num_objs+1):
        for key in ['landmarks', 'bounding_box', 'category_id', 'occlusion', 'category_name']:
            anno[k-1][key] = raw_anno["item{}".format(k)][key]
    annos.append(anno)
    break

In [6]:
annos[0]

defaultdict(dict,
            {0: {'landmarks': [238,
               195,
               1,
               219,
               189,
               1,
               225,
               197,
               1,
               238,
               200,
               2,
               251,
               197,
               1,
               260,
               190,
               1,
               201,
               202,
               2,
               213,
               218,
               1,
               213,
               239,
               1,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               0,
               274,
               243,
               2,
               282,
               223,
               2,
               287,
               196,
               2],
              'bounding_box': [199, 190, 287, 269],
              'category_id': 5,
              'occlusio

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pycocotools.coco as coco
from pycocotools.cocoeval import COCOeval
import numpy as np
import json
import os

import torch.utils.data as data

class DeepFashion2KP(data.Dataset):
    def __init__(self, coco_handle):
        self.coco = coco_handle
        cats = self.coco.loadCats(self.coco.getCatIds())
        self.num_classes = len(cats)
    
    flip_idx = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], 
                [11, 12], [13, 14], [15, 16]]
    def __init__(self, opt, split):
      super(COCOHP, self).__init__()
      self.edges = [[0, 1], [0, 2], [1, 3], [2, 4], 
                    [4, 6], [3, 5], [5, 6], 
                    [5, 7], [7, 9], [6, 8], [8, 10], 
                    [6, 12], [5, 11], [11, 12], 
                    [12, 14], [14, 16], [11, 13], [13, 15]]
      
      self.acc_idxs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
      self.data_dir = os.path.join(opt.data_dir, 'coco')
      self.img_dir = os.path.join(self.data_dir, '{}2017'.format(split))
      if split == 'test':
        self.annot_path = os.path.join(
            self.data_dir, 'annotations', 
            'image_info_test-dev2017.json').format(split)
      else:
        self.annot_path = os.path.join(
          self.data_dir, 'annotations', 
          'person_keypoints_{}2017.json').format(split)
      self.max_objs = 32
      self._data_rng = np.random.RandomState(123)
      self._eig_val = np.array([0.2141788, 0.01817699, 0.00341571],
                               dtype=np.float32)
      self._eig_vec = np.array([
          [-0.58752847, -0.69563484, 0.41340352],
          [-0.5832747, 0.00994535, -0.81221408],
          [-0.56089297, 0.71832671, 0.41158938]
      ], dtype=np.float32)
      self.split = split
      self.opt = opt
  
      print('==> initializing coco 2017 {} data.'.format(split))
      self.coco = coco.COCO(self.annot_path)
      image_ids = self.coco.getImgIds()
  
      if split == 'train':
        self.images = []
        for img_id in image_ids:
          idxs = self.coco.getAnnIds(imgIds=[img_id])
          if len(idxs) > 0:
            self.images.append(img_id)
      else:
        self.images = image_ids
      self.num_samples = len(self.images)
      print('Loaded {} {} samples'.format(split, self.num_samples))
  
    def _to_float(self, x):
      return float("{:.2f}".format(x))
  
    def convert_eval_format(self, all_bboxes):
      # import pdb; pdb.set_trace()
      detections = []
      for image_id in all_bboxes:
        for cls_ind in all_bboxes[image_id]:
          category_id = 1
          for dets in all_bboxes[image_id][cls_ind]:
            bbox = dets[:4]
            bbox[2] -= bbox[0]
            bbox[3] -= bbox[1]
            score = dets[4]
            bbox_out  = list(map(self._to_float, bbox))
            keypoints = np.concatenate([
              np.array(dets[5:39], dtype=np.float32).reshape(-1, 2), 
              np.ones((17, 1), dtype=np.float32)], axis=1).reshape(51).tolist()
            keypoints  = list(map(self._to_float, keypoints))
  
            detection = {
                "image_id": int(image_id),
                "category_id": int(category_id),
                "bbox": bbox_out,
                "score": float("{:.2f}".format(score)),
                "keypoints": keypoints
            }
            detections.append(detection)
      return detections
  
    def __len__(self):
      return self.num_samples
  
    def save_results(self, results, save_dir):
      json.dump(self.convert_eval_format(results), 
                open('{}/results.json'.format(save_dir), 'w'))
  
  
    def run_eval(self, results, save_dir):
      # result_json = os.path.join(opt.save_dir, "results.json")
      # detections  = convert_eval_format(all_boxes)
      # json.dump(detections, open(result_json, "w"))
      self.save_results(results, save_dir)
      coco_dets = self.coco.loadRes('{}/results.json'.format(save_dir))
      coco_eval = COCOeval(self.coco, coco_dets, "keypoints")
      coco_eval.evaluate()
      coco_eval.accumulate()
      coco_eval.summarize()
      coco_eval = COCOeval(self.coco, coco_dets, "bbox")
      coco_eval.evaluate()
      coco_eval.accumulate()
      coco_eval.summarize()